In [4]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import PandasTools

# Cargar los datos
print("Cargando bases de datos...")

# Cargar el archivo TSV
tsv_file = "/home/edwin/proyectos/spent_coffee_grounds/lipids/data_mining/databases/databases_orig/COMP_DB_DATA.tsv"
df_tsv = pd.read_csv(tsv_file, sep='\t')

# Cargar el archivo SDF
sdf_file = "/home/edwin/proyectos/spent_coffee_grounds/lipids/data_mining/databases/databases_orig/structures.sdf"
df_sdf = PandasTools.LoadSDF(sdf_file)

print(f"Compuestos en TSV: {len(df_tsv)}")
print(f"Compuestos en SDF: {len(df_sdf)}")

# Mostrar todas las columnas para referencia
print("\nColumnas en TSV:")
print(df_tsv.columns.tolist())
print("\nColumnas en SDF:")
print(df_sdf.columns.tolist())

# Usar 'abbrev' del TSV y 'ABBREVIATION' del SDF como columnas comunes
tsv_ids = set(df_tsv['abbrev'].dropna().astype(str))
sdf_ids = set(df_sdf['ABBREVIATION'].dropna().astype(str))

print(f"\nAbreviaturas únicas en TSV: {len(tsv_ids)}")
print(f"Abreviaturas únicas en SDF: {len(sdf_ids)}")

# Encontrar compuestos en TSV pero no en SDF
compuestos_exclusivos_tsv = tsv_ids - sdf_ids

print(f"\nCompuestos exclusivos del TSV: {len(compuestos_exclusivos_tsv)}")

# Filtrar el DataFrame TSV para obtener solo los compuestos exclusivos
df_exclusivos = df_tsv[df_tsv['abbrev'].astype(str).isin(compuestos_exclusivos_tsv)]

# Guardar la base de datos exclusiva
output_file = "/home/edwin/proyectos/spent_coffee_grounds/lipids/data_mining/databases/compuestos_exclusivos_tsv.tsv"
df_exclusivos.to_csv(output_file, sep='\t', index=False)

print(f"\nBase de datos de compuestos exclusivos guardada en: {output_file}")
print(f"Compuestos guardados: {len(df_exclusivos)}")

# Mostrar algunos ejemplos de compuestos exclusivos
print("\nPrimeros 10 compuestos exclusivos del TSV:")
print(list(compuestos_exclusivos_tsv)[:10])

# Información adicional sobre la base de datos creada
print(f"\nInformación de la base de datos creada:")
print(f"Número de compuestos: {len(df_exclusivos)}")
print(f"Columnas: {df_exclusivos.columns.tolist()}")
print(f"\nPrimeras 5 filas:")
print(df_exclusivos.head())

Cargando bases de datos...


[13:34:17] Warning: ambiguous stereochemistry - zero final chiral volume - at atom 5 ignored
[13:34:17] Warning: ambiguous stereochemistry - zero final chiral volume - at atom 5 ignored
[13:34:17] Warning: ambiguous stereochemistry - zero final chiral volume - at atom 5 ignored
[13:34:17] Warning: ambiguous stereochemistry - zero final chiral volume - at atom 5 ignored
[13:34:17] Warning: ambiguous stereochemistry - zero final chiral volume - at atom 5 ignored
[13:34:17] Warning: ambiguous stereochemistry - opposing bonds have opposite wedging - at atom 7 ignored.
[13:34:17] Warning: ambiguous stereochemistry - opposing bonds have opposite wedging - at atom 7 ignored.
[13:34:17] Warning: ambiguous stereochemistry - opposing bonds have opposite wedging - at atom 3 ignored.
[13:34:17] Warning: ambiguous stereochemistry - opposing bonds have opposite wedging - at atom 3 ignored.
[13:34:18] Warning: conflicting stereochemistry - bond wedging contradiction - at atom 49 ignored
[13:34:18] Wa

Compuestos en TSV: 64891
Compuestos en SDF: 49267

Columnas en TSV:
['headgroup', 'abbrev', 'mass', 'formula', 'chain_type']

Columnas en SDF:
['LM_ID', 'NAME', 'SYSTEMATIC_NAME', 'CATEGORY', 'MAIN_CLASS', 'EXACT_MASS', 'FORMULA', 'INCHI_KEY', 'INCHI', 'SMILES', 'ABBREVIATION', 'SYNONYMS', 'PUBCHEM_CID', 'CHEBI_ID', 'ID', 'ROMol', 'KEGG_ID', 'HMDB_ID', 'SWISSLIPIDS_ID', 'SUB_CLASS', 'LIPIDBANK_ID', 'PLANTFA_ID', 'CLASS_LEVEL4']

Abreviaturas únicas en TSV: 64891
Abreviaturas únicas en SDF: 6892

Compuestos exclusivos del TSV: 60038

Base de datos de compuestos exclusivos guardada en: /home/edwin/proyectos/spent_coffee_grounds/lipids/data_mining/databases/compuestos_exclusivos_tsv.tsv
Compuestos guardados: 60038

Primeros 10 compuestos exclusivos del TSV:
['PT O-68:5', 'Cer 45:3;O5', 'LPT O-19:1;O', 'PI 67:5;O', 'ST 22:6;O7;Tau', 'LPE O-30:0', 'PT O-25:6;O', 'CL 68:7', 'ST 22:0;O4', 'TG O-85:19']

Información de la base de datos creada:
Número de compuestos: 60038
Columnas: ['headgroup'

In [5]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import PandasTools

def encontrar_compuestos_exclusivos(df_tsv, df_sdf):
    """Encuentra compuestos en TSV pero no en SDF probando diferentes columnas"""
    
    # Posibles pares de columnas para comparar
    pares_columnas = [
        ('abbrev', 'ABBREVIATION'),
        ('formula', 'FORMULA'),
        ('headgroup', 'CATEGORY'),
        ('headgroup', 'MAIN_CLASS')
    ]
    
    for col_tsv, col_sdf in pares_columnas:
        if col_tsv in df_tsv.columns and col_sdf in df_sdf.columns:
            print(f"Probando con {col_tsv} (TSV) vs {col_sdf} (SDF)")
            
            tsv_ids = set(df_tsv[col_tsv].dropna().astype(str))
            sdf_ids = set(df_sdf[col_sdf].dropna().astype(str))
            
            compuestos_exclusivos = tsv_ids - sdf_ids
            
            print(f"  - Compuestos exclusivos encontrados: {len(compuestos_exclusivos)}")
            
            if len(compuestos_exclusivos) > 0:
                return df_tsv[df_tsv[col_tsv].astype(str).isin(compuestos_exclusivos)], compuestos_exclusivos
    
    # Si ninguna columna funciona, devolver DataFrame vacío
    return pd.DataFrame(), set()

# Cargar datos
print("Cargando bases de datos...")
tsv_file = "/home/edwin/proyectos/spent_coffee_grounds/lipids/data_mining/databases/databases_orig/COMP_DB_DATA.tsv"
sdf_file = "/home/edwin/proyectos/spent_coffee_grounds/lipids/data_mining/databases/databases_orig/structures.sdf"

df_tsv = pd.read_csv(tsv_file, sep='\t')
df_sdf = PandasTools.LoadSDF(sdf_file)

print(f"TSV: {len(df_tsv)} compuestos")
print(f"SDF: {len(df_sdf)} compuestos")

# Encontrar compuestos exclusivos
df_exclusivos, compuestos_exclusivos = encontrar_compuestos_exclusivos(df_tsv, df_sdf)

if len(df_exclusivos) > 0:
    # Guardar resultados
    output_file = "/home/edwin/proyectos/spent_coffee_grounds/lipids/data_mining/databases/compuestos_exclusivos_tsv.tsv"
    df_exclusivos.to_csv(output_file, sep='\t', index=False)
    
    print(f"\nBase de datos guardada en: {output_file}")
    print(f"Total de compuestos exclusivos guardados: {len(df_exclusivos)}")
    
    # Mostrar resumen
    print("\nEjemplos de compuestos exclusivos:")
    for i, comp_id in enumerate(list(compuestos_exclusivos)[:10]):
        print(f"  {i+1}. {comp_id}")
else:
    print("\nNo se encontraron compuestos exclusivos o no se pudo determinar la columna común.")
    print("\nColumnas en TSV:", df_tsv.columns.tolist())
    print("Columnas en SDF:", df_sdf.columns.tolist())

Cargando bases de datos...


[13:41:50] Warning: ambiguous stereochemistry - zero final chiral volume - at atom 5 ignored
[13:41:50] Warning: ambiguous stereochemistry - zero final chiral volume - at atom 5 ignored
[13:41:50] Warning: ambiguous stereochemistry - zero final chiral volume - at atom 5 ignored
[13:41:50] Warning: ambiguous stereochemistry - zero final chiral volume - at atom 5 ignored
[13:41:50] Warning: ambiguous stereochemistry - zero final chiral volume - at atom 5 ignored
[13:41:50] Warning: ambiguous stereochemistry - opposing bonds have opposite wedging - at atom 7 ignored.
[13:41:50] Warning: ambiguous stereochemistry - opposing bonds have opposite wedging - at atom 7 ignored.
[13:41:50] Warning: ambiguous stereochemistry - opposing bonds have opposite wedging - at atom 3 ignored.
[13:41:50] Warning: ambiguous stereochemistry - opposing bonds have opposite wedging - at atom 3 ignored.
[13:41:51] Warning: conflicting stereochemistry - bond wedging contradiction - at atom 49 ignored
[13:41:51] Wa

TSV: 64891 compuestos
SDF: 49267 compuestos
Probando con abbrev (TSV) vs ABBREVIATION (SDF)
  - Compuestos exclusivos encontrados: 60038

Base de datos guardada en: /home/edwin/proyectos/spent_coffee_grounds/lipids/data_mining/databases/compuestos_exclusivos_tsv.tsv
Total de compuestos exclusivos guardados: 60038

Ejemplos de compuestos exclusivos:
  1. PT O-68:5
  2. Cer 45:3;O5
  3. LPT O-19:1;O
  4. PI 67:5;O
  5. ST 22:6;O7;Tau
  6. LPE O-30:0
  7. PT O-25:6;O
  8. CL 68:7
  9. ST 22:0;O4
  10. TG O-85:19
